# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("OPAvanzadas")\
    .getOrCreate()

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [4]:
(cleanFlightsDF.select(F.col("Origin"),F.col("Dest"),F.col("ArrDelay"))\
                       .groupBy(F.col("Origin"),F.col("Dest"))\
                       .agg(avg(F.col("ArrDelay")).alias("retrasoMedio"),count("*").alias("numVuelos"))
                       .sort(F.col("retrasoMedio").desc())
                       .show(20))

+------+----+------------------+---------+
|Origin|Dest|      retrasoMedio|numVuelos|
+------+----+------------------+---------+
|   RDM| MFR|            1347.0|        2|
|   MDT| HPN|             798.0|        1|
|   ORD| GTF|             212.0|        1|
|   ICT| DAY|             210.0|        1|
|   ELM| ATL|             169.0|        2|
|   DSM| PIA|             168.0|        1|
|   MSP| LEX|             153.0|        2|
|   PPG| HNL|144.22222222222223|       18|
|   HNL| PPG|143.05555555555554|       18|
|   YNG| PIE|             141.0|        1|
|   BUF| MSP|             128.0|        2|
|   CMH| HOU|             120.0|        1|
|   HRL| DAL|             111.0|        1|
|   PIE| YNG|             104.0|        1|
|   AVP| SFB|              93.0|        1|
|   DFW| LIH| 91.71428571428571|       15|
|   OWB| SFB|           90.5625|       19|
|   CPR| LAS|              85.0|        1|
|   TWF| ORD|              83.0|        1|
|   LAS| CPR|              82.0|        1|
+------+---

In [6]:
cleanFlightsDF.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- ArrDelayCat: string (nullable 

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [36]:
# UTilizamos pivot para coger todos los valores posibles de "ArrDelayCat".
# Cada posible valor se transforma en una columna
# Le agregamos un conteo para cada tipo y obtenemos el máximo para cada tipo también
# Después lo ordenamos por origen (por ningún motivo real)

pivot = cleanFlightsDF.groupBy("Origin")\
                      .pivot("ArrDelayCat").agg(
                          F.count("*").alias("Conteo"),
                          F.max("ArrDelay").alias("maxArrDelay"))\
                        .sort("Origin")
pivot.show()

+------+-----------+----------------+-----------+----------------+-------------+------------------+
|Origin|Huge_Conteo|Huge_maxArrDelay|None_Conteo|None_maxArrDelay|Slight_Conteo|Slight_maxArrDelay|
+------+-----------+----------------+-----------+----------------+-------------+------------------+
|   ABE|         97|           674.0|        554|            14.0|          112|              59.0|
|   ABI|         37|           397.0|        229|            14.0|           46|              59.0|
|   ABQ|        242|          1073.0|       3368|            14.0|          434|              59.0|
|   ABR|         16|           584.0|         95|            13.0|            7|              33.0|
|   ABY|         20|           641.0|        117|            14.0|           23|              48.0|
|   ACT|         23|           424.0|        170|            14.0|           22|              59.0|
|   ACV|         18|           255.0|        147|            14.0|           16|              59.0|


<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

In [17]:
(cleanFlightsDF
 .groupBy(F.col("DayOfWeek"))\
 .agg(F.mean("ArrDelay"),F.mean("DepDelay"))\
 .sort("DayOfWeek")\
 .show())

+---------+--------------------+------------------+
|DayOfWeek|       avg(ArrDelay)|     avg(DepDelay)|
+---------+--------------------+------------------+
|        1|   5.862438897075216| 11.32756264236902|
|        2|   2.135074656089688| 8.522952978196345|
|        3|   3.218429729970591| 9.017015081296748|
|        4|   2.091846182334424|  8.39038196495373|
|        5|    5.97030310020579|11.769760656650647|
|        6|-0.09118143102478996|  7.10215602989408|
|        7|   5.152362335166655|11.203569495597534|
+---------+--------------------+------------------+



In [19]:
pandas = (cleanFlightsDF.groupBy(F.col("Origin"))\
 .pivot("DayOfWeek")\
 .agg(F.mean("ArrDelay"),F.mean("DepDelay"))\
 .sort("Origin")\
 .toPandas())

pandas

,Origin,1_avg(ArrDelay),1_avg(DepDelay),2_avg(ArrDelay),2_avg(DepDelay),3_avg(ArrDelay),3_avg(DepDelay),4_avg(ArrDelay),4_avg(DepDelay),5_avg(ArrDelay),5_avg(DepDelay),6_avg(ArrDelay),6_avg(DepDelay),7_avg(ArrDelay),7_avg(DepDelay)
0,ABE,19.512397,19.537190,22.776699,24.019417,15.256410,16.119658,8.037037,12.555556,16.581818,21.890909,8.360465,9.500000,21.042105,20.656250
1,ABI,21.000000,22.960784,27.727273,26.272727,7.325000,6.150000,2.363636,4.409091,8.111111,8.177778,8.033333,8.466667,7.444444,9.800000
2,ABQ,2.609034,10.524922,0.129508,7.241042,-0.380719,6.859477,0.090753,6.832765,-1.865854,5.296167,-4.571429,4.402715,-2.176806,6.283554
3,ABR,17.411765,17.529412,10.000000,11.375000,14.111111,21.555556,38.562500,39.812500,25.750000,32.562500,10.933333,8.333333,11.875000,21.125000
4,ABY,11.923077,15.000000,0.041667,3.434783,19.272727,13.173913,18.826087,15.545455,-0.826087,-3.136364,30.600000,28.750000,49.350000,51.947368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,XNA,11.771341,15.024169,12.138037,19.600000,10.859281,13.627976,2.974277,7.535256,8.895425,12.824104,3.131579,5.598684,9.015152,8.754717
349,YAK,-16.846154,-13.500000,-16.333333,-19.500000,-17.444444,-17.111111,-9.600000,-17.066667,-21.187500,-21.187500,-21.000000,-21.812500,-13.666667,-16.866667
350,YKM,-1.068966,-0.758621,0.166667,-1.600000,10.812500,11.593750,0.678571,-1.250000,2.592593,-1.222222,6.125000,6.583333,3.185185,3.296296
351,YNG,9.000000,15.000000,NaN,NaN,NaN,NaN,141.000000,111.000000,NaN,NaN,NaN,NaN,NaN,NaN


<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

In [20]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest").agg(
    F.mean("ArrDelay").alias("avgArrDelay")
)

In [5]:
preguntaFour = averageDelayOriginDestDF.join(cleanFlightsDF,
on =
(averageDelayOriginDestDF["Origin"] == cleanFlightsDF["Origin"]) &\
(averageDelayOriginDestDF["Dest"] == cleanFlightsDF["Dest"]))
preguntaFour.select(averageDelayOriginDestDF["Origin"], \
averageDelayOriginDestDF["Dest"], "DayOfWeek", "ArrDelay", "avgArrDelay").show(5)

NameError: name 'averageDelayOriginDestDF' is not defined

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [4]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [8]:
joinedFlightsDF = cleanFlightsDF.join(airportsDF,
                                      on = cleanFlightsDF["Origin"] == airportsDF["IATA"],
                                      how = "left_outer")

# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.filter(~(F.col("FullName").isNull()))\
                .select("Origin","Dest","IATA","FullName","Year")\
                .show(10, truncate = False)

+------+----+----+------------------------------------+----+
|Origin|Dest|IATA|FullName                            |Year|
+------+----+----+------------------------------------+----+
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |JFK |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |BOS |SEA |Seattle-Tacoma International Airport|1949|
|SEA   |LGB |SEA |Seattle-Tacoma International Airport|1949|
+------+----+----+------------------------------------+----+
only showing top 10 rows



## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 596, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 200, in _batched
    for item in iterator:
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 450, in mapper
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 450, in <genexpr>
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 85, in <lambda>
  File "C:\spark\python\lib\pyspark.zip\pyspark\util.py", line 73, in wrapper

<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [12]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    #return ????
    
# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = 

# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = 

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()

+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   SEA| JFK|        2|        Tuesday|
|   DFW| FLL|        7|         Sunday|
|   PHL| SJU|        1|         Monday|
|   GEG| DFW|        6|       Saturday|
|   OAK| PHX|        2|        Tuesday|
|   LAX| DEN|        7|         Sunday|
|   LAX| SEA|        1|         Monday|
|   OAK| PDX|        4|       Thursday|
|   LAS| PDX|        5|         Friday|
|   SLC| PDX|        5|         Friday|
|   SJC| LIH|        6|       Saturday|
|   SJC| EWR|        7|         Sunday|
|   KOA| ANC|        7|         Sunday|
|   SEA| DTW|        1|         Monday|
|   DTW| SEA|        1|         Monday|
|   SEA| BOS|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   ATL| XNA|        5|         Friday|
|   OAK| SLC|        5|         Friday|
|   MSP| ANC|        5|         Friday|
+------+----+---------+---------------+
only showing top 20 rows

